Set your parameters

In [1]:
# What is your major?
MAJOR_SUPPORT = "CISC"
# Do you need FE courses recommendation?
FE_SUPPORT = False
# What is the academic year now?
YEAR_SUPPORT = "2024a"
# How many suggestion do you want?
SUGGESTION_NUM = 30

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

Convert your timetable into special data structure. You can use AI tools like GPT-4 or Claude-3.5 to accomplish this. In this case, you may need to use the following prompt:
```
my_timetable = [
{
"course_code": "GEST1009",
"day": "MON",
"start_time": "08:30",
"end_time": "09:45",
"type": "Lecture"
},
{
"course_code": "GEST1009",
"day": "THU",
"start_time": "08:30",
"end_time": "09:45",
"type": "Lecture"
},
{
"course_code": "CISC1002",
"day": "MON",
"start_time": "11:30",
"end_time": "12:45",
"type": "Lecture"
},
# ... (continue for all other classes)
]

Convert all the data from the timetable image into the format shown above.
```

In [4]:
timetable = [
    {
        "course_code": "GEST1009",
        "day": "MON",
        "start_time": "08:30",
        "end_time": "09:45",
        "type": "Lecture"
    },
    {
        "course_code": "GEST1009",
        "day": "THU",
        "start_time": "08:30",
        "end_time": "09:45",
        "type": "Lecture"
    },
    {
        "course_code": "CISC1002",
        "day": "MON",
        "start_time": "11:30",
        "end_time": "12:45",
        "type": "Lecture"
    },
    {
        "course_code": "CISC1002",
        "day": "THU",
        "start_time": "11:30",
        "end_time": "12:45",
        "type": "Lecture"
    },
    {
        "course_code": "CISC1002",
        "day": "FRI",
        "start_time": "10:00",
        "end_time": "11:15",
        "type": "Lab/Tutorial"
    },
    {
        "course_code": "GEGA1000",
        "day": "TUE",
        "start_time": "14:30",
        "end_time": "15:45",
        "type": "Lecture"
    },
    {
        "course_code": "GEGA1000",
        "day": "FRI",
        "start_time": "14:30",
        "end_time": "15:45",
        "type": "Lecture"
    },
    {
        "course_code": "CISC2001",
        "day": "MON",
        "start_time": "16:00",
        "end_time": "17:15",
        "type": "Lecture"
    },
    {
        "course_code": "CISC2001",
        "day": "THU",
        "start_time": "16:00",
        "end_time": "17:15",
        "type": "Lecture"
    },
    {
        "course_code": "CISC2001",
        "day": "FRI",
        "start_time": "17:30",
        "end_time": "18:45",
        "type": "Lab/Tutorial"
    },
    {
        "course_code": "CISC2003",
        "day": "TUE",
        "start_time": "16:00",
        "end_time": "17:15",
        "type": "Lecture"
    },
    {
        "course_code": "CISC2003",
        "day": "WED",
        "start_time": "16:00",
        "end_time": "17:15",
        "type": "Lab/Tutorial"
    },
    {
        "course_code": "CISC2003",
        "day": "FRI",
        "start_time": "16:00",
        "end_time": "17:15",
        "type": "Lecture"
    },
    {
        "course_code": "HONR2003",
        "day": "WED",
        "start_time": "18:30",
        "end_time": "21:15",
        "type": "Lecture"
    }
]

In [5]:
cm = pd.read_csv("2024a/cm.csv").dropna(axis=0)
ge = pd.read_csv("2024a/ge.csv").dropna(axis=0)

In [6]:
if FE_SUPPORT == True:
    df = pd.concat([cm, ge], ignore_index=True)
else:
    cm = cm[cm['course_name'].str.startswith(f'{MAJOR_SUPPORT}', na=False)]
    df = pd.concat([cm, ge], ignore_index=True)

In [7]:
grade_order = {
    'A+': 12, 'A': 11, 'A-': 10,
    'B+': 9, 'B': 8, 'B-': 7,
    'C+': 6, 'C': 5, 'C-': 4,
    'D+': 3, 'D': 2, 'D-': 1,
    'F': 0
}

def grade_to_numeric(grade):
    return grade_order.get(grade, -1) 

df = df.sort_values(
    by=['overall_grade', 'course_name', 'name'],
    key=lambda x: x.map(lambda y: grade_to_numeric(y) if x.name == 'overall_grade' else y),
    ascending=[False, True, True]
)

In [8]:
def parse_time(time_str):
    return datetime.strptime(time_str, '%H:%M')

def check_conflict(existing_class, new_class):
    if existing_class['day'] != new_class['day']:
        return False
    
    existing_start = parse_time(existing_class['start_time'])
    existing_end = parse_time(existing_class['end_time'])
    new_start = parse_time(new_class['start_time'])
    new_end = parse_time(new_class['end_time'])
    
    return (existing_start <= new_start < existing_end) or \
           (existing_start < new_end <= existing_end) or \
           (new_start <= existing_start and existing_end <= new_end)

def parse_course_time(time_str):
    time_slots = []
    for slot in time_str.split('&'):
        day, time_range = slot.strip().split(' ', 1)
        start_time, end_time = time_range.split('-')
        time_slots.append({
            'day': day,
            'start_time': start_time,
            'end_time': end_time
        })
    return time_slots

def has_conflict_with_timetable(course_time, my_timetable):
    course_slots = parse_course_time(course_time)
    for new_slot in course_slots:
        for existing_class in my_timetable:
            if check_conflict(existing_class, new_slot):
                return True
    return False

In [9]:
def show(df, timetable):
    non_conflicting_courses = df[~df['time'].apply(lambda x: has_conflict_with_timetable(x, timetable))]
    print(non_conflicting_courses[:SUGGESTION_NUM].to_string(index=False))

In [10]:
df


,course_name,name,section,overall_grade,grade,difficulty,useful,comments,time
0,CISC3001,WU YUAN,1,A,A,A,A,3,MON 17:30-18:45 & THU 17:30-18:45 & WED 10:00-...
16,GELH1001,CHEONG IM LAN,3,A,A,A,A,10,MON 14:30-15:45 & THU 14:30-15:45
17,GELH1001,CHEONG IM LAN,4,A,A,A,A,10,MON 16:00-17:15 & THU 16:00-17:15
18,GELH1007,LI ZIHAO,1,A,A,A,A,2,SAT 08:30-11:15
19,GEST2002,KUZMA STRELNIKOV,1,A,A,A,A,2,MON 11:30-12:45 & THU 11:30-12:45
20,GELH1000,SUN XIAOXUE,10,A-,A,A,A,1,MON 08:30-09:45 & THU 08:30-09:45
21,GELH1010,LEE TING MIEN,1,A-,A,A,A,1,FRI 08:30-09:45 & TUE 08:30-09:45
1,CISC3014,WANG PENGYANG,1,B+,B,B,B+,7,MON 14:30-15:45 & THU 14:30-15:45
22,GEGA1006,ZHA CHENYANG,4,B+,A-,A-,A-,18,FRI 17:30-18:45 & TUE 17:30-18:45
23,GELH2001,JEREMY CENTENO DE CHAVEZ,1,B+,A,B+,B+,18,MON 10:00-12:45


In [12]:
def style_dataframe(df):
    def highlight_grades(val):
        color_map = {
            'A': '#4CAF50',  # 绿色
            'A-': '#8BC34A',
            'B+': '#2196F3', # 蓝色
            'B': '#03A9F4',
            'B-': '#00BCD4',
            'C+': '#FFC107', # 琥珀色
            'C': '#FFB300',
            'C-': '#FFA000',
            'D': '#FF5722',  # 深橙色
            'F': '#F44336'   # 红色
        }
        return f'color: {color_map.get(val, "#212529")}; font-weight: bold;'

    return (df.style
            .set_properties(**{
                'background-color': '#f8f9fa',
                'color': '#212529',
                'border-color': '#dee2e6',
                'font-family': 'Arial, sans-serif',
                'font-size': '12px',
                'text-align': 'left',
                'white-space': 'normal',  # 允许文本换行
                'max-width': '150px'
            })
            .set_table_styles([
                {'selector': 'th', 'props': [
                    ('background-color', '#e9ecef'),
                    ('color', '#495057'),
                    ('font-weight', 'bold'),
                    ('text-align', 'left'),
                    ('padding', '12px'),
                    ('border-bottom', '2px solid #dee2e6')
                ]},
                {'selector': 'td', 'props': [('padding', '12px')]},
                {'selector': 'tr:nth-of-type(even)', 'props': [('background-color', '#ffffff')]},
            ])
            .map(highlight_grades, subset=['overall_grade', 'grade'])
            .highlight_max(subset=['difficulty', 'useful'], color='#fce4ec')
            .highlight_min(subset=['difficulty', 'useful'], color='#e8f5e9')
           )
def show():
    styled_df = style_dataframe(df)
    display(styled_df)

In [13]:
show()

,course_name,name,section,overall_grade,grade,difficulty,useful,comments,time
0,CISC3001,WU YUAN,1,A,A,A,A,3,MON 17:30-18:45 & THU 17:30-18:45 & WED 10:00-11:15
16,GELH1001,CHEONG IM LAN,3,A,A,A,A,10,MON 14:30-15:45 & THU 14:30-15:45
17,GELH1001,CHEONG IM LAN,4,A,A,A,A,10,MON 16:00-17:15 & THU 16:00-17:15
18,GELH1007,LI ZIHAO,1,A,A,A,A,2,SAT 08:30-11:15
19,GEST2002,KUZMA STRELNIKOV,1,A,A,A,A,2,MON 11:30-12:45 & THU 11:30-12:45
20,GELH1000,SUN XIAOXUE,10,A-,A,A,A,1,MON 08:30-09:45 & THU 08:30-09:45
21,GELH1010,LEE TING MIEN,1,A-,A,A,A,1,FRI 08:30-09:45 & TUE 08:30-09:45
1,CISC3014,WANG PENGYANG,1,B+,B,B,B+,7,MON 14:30-15:45 & THU 14:30-15:45
22,GEGA1006,ZHA CHENYANG,4,B+,A-,A-,A-,18,FRI 17:30-18:45 & TUE 17:30-18:45
23,GELH2001,JEREMY CENTENO DE CHAVEZ,1,B+,A,B+,B+,18,MON 10:00-12:45
